In [5]:
################################################################################
# MASTERARBEIT - SKRIPT 07:
# EXPERIMENT 2.1 (Forschungsfrage 2) - INKONSISTENZBEHEBUNG (HoloClean)
################################################################################
#
# ZWECK DIESES SKRIPTS (Methodik gemäß Abschnitt 3.3.2):
#
# 1. (Laden): Lädt den 'schmutzigen' Rohdatensatz (rfd_main.csv).
# 2. (Methode): Wendet HoloClean als erste Methode zur Inkonsistenzbehebung an.
# 3. (Ziel): Reparatur syntaktischer und logischer/semantischer Fehler.
# 4. (Constraints): Lädt rfd_constraints.txt (Regeln/FDs).
# 5. (Speichern): Schreibt reparierte Daten als CSV.
# 6. (Umgebung): Python 3.7.x kompatibel zu HoloClean.
#
################################################################################

# Schritt 1: Notwendige Bibliotheken importieren
import sys
import os
import pandas as pd
from sqlalchemy import create_engine
import logging
import time

# --- HoloClean-Pfad hinzufügen (falls lokal geklont) ---
HoloClean_pfad = os.path.join(os.getcwd(), 'holoclean')
if HoloClean_pfad not in sys.path:
    sys.path.append(HoloClean_pfad)
# --- Ende Modulpfad-Korrektur ---

print("Lade HoloClean-Bibliotheken...")
try:
    from holoclean.holoclean import HoloClean
    from holoclean.detect.nulldetector import NullDetector
    from holoclean.detect.violationdetector import ViolationDetector
    from holoclean.repair.featurize import (
        InitAttrFeaturizer, OccurAttrFeaturizer, FreqFeaturizer, ConstraintFeaturizer
    )
    print("HoloClean-Module erfolgreich geladen.")
except ImportError as e:
    print("FEHLER: HoloClean-Bibliotheken konnten nicht geladen werden.")
    print(f"Stellen Sie sicher, dass HoloClean im Ordner '{HoloClean_pfad}' liegt"
          " UND eine passende Python 3.7.x-Umgebung aktiv ist.")
    print(f"Fehlermeldung: {e}")
    sys.exit("Skript gestoppt, da Abhängigkeiten fehlen.")
except Exception as e:
    print(f"Ein unerwarteter Fehler beim Import ist aufgetreten: {e}")
    sys.exit("Skript gestoppt.")

print("Alle Bibliotheken sind bereit.")
print("=" * 70)

if __name__ == '__main__':
    print("Starte HoloClean-Hauptprozess...")

    # Konstanten / Pfade
    pfad_daten = 'rfd_main.csv'
    pfad_constraints = 'rfd_constraints.txt'
    tabellename = 'rfd_main'
    ausgabedatei = 'ergebnisse/2.1_rfd_repaired_holoclean.csv'

    # PostgreSQL (Docker)
    DB_NAME = "holo"
    DB_USER = "holocleanuser"
    DB_PASS = "abcd1234"
    DB_HOST = "127.0.0.1"
    DB_PORT = 5432

    os.makedirs('ergebnisse', exist_ok=True)
    logging.basicConfig(level=logging.INFO)

    # 1) HoloClean-Session
    try:
        hc = HoloClean(
            db_name=DB_NAME, username=DB_USER, password=DB_PASS,
            host=DB_HOST, port=DB_PORT, threads=1, verbose=False, timeout=3*60000
        ).session
        print(f"HoloClean-Session erfolgreich initialisiert (DB: '{DB_NAME}').")
    except Exception as e:
        print(f"FEHLER: HoloClean-Session konnte nicht initialisiert werden. {e}")
        print("Prüfen Sie, ob PostgreSQL (Docker) läuft und Zugangsdaten stimmen.")
        sys.exit("Skript gestoppt.")

    # --- SCHRITT 2: Robuster Daten- und Constraints-Ladevorgang (KeyError-KORREKTUR) ---
    print("\n--- Schritt 2: Daten laden (KeyError-Korrektur) ---")

    # METHODISCHE KORREKTUR: URL-Spalte ausschließen + lange Tokens maskieren
    AUSZUSCHLIESSENDE_SPALTEN = ['url']  # Nur die URL-Spalte initial ausschließen

    try:
        # 1) Daten in Pandas laden und PROBLEM-SPALTEN entfernen
        print(f"1) Lade Rohdaten aus '{pfad_daten}' und entferne Spalten: {AUSZUSCHLIESSENDE_SPALTEN} ...")
        df_raw = pd.read_csv(pfad_daten)
        df_hc = df_raw.drop(columns=AUSZUSCHLIESSENDE_SPALTEN, errors='ignore')

        # 2) Optional: URL-ähnliche Tokens und extrem lange Strings maskieren
        import re, numpy as np
        url_regex = re.compile(r'https?://\S+', flags=re.IGNORECASE)

        def _mask_problemwerte(x):
            if isinstance(x, str):
                if url_regex.search(x):
                    return np.nan
                if len(x) > 300:
                    return np.nan
            return x

        for col in df_hc.columns:
            if df_hc[col].dtype == object:
                df_hc[col] = df_hc[col].map(_mask_problemwerte)

        # 3) Bereinigten Datensatz als temporäre CSV für HoloClean schreiben
        pfad_daten_hc = "rfd_main_for_hc.csv"
        df_hc.to_csv(pfad_daten_hc, index=False)
        print(f"2) Schreibe bereinigten Datensatz nach '{pfad_daten_hc}'.")

        # 4) Daten mit HoloClean laden
        print(f"3) Lade Daten in Tabelle '{tabellename}' über HoloClean (load_data) ...")
        hc.load_data(tabellename, pfad_daten_hc)

        # 5) Constraints laden und registrieren
        print(f"4) Lade Constraints (Regeln) aus '{pfad_constraints}' ...")
        hc.load_dcs(pfad_constraints)
        hc.ds.set_constraints(hc.get_dcs())

        print("Daten (ohne 'url') und Constraints wurden erfolgreich geladen und verknüpft.")

    except Exception as e:
        print(f"FEHLER beim Laden der Daten oder der Constraints: {e}")
        sys.exit("Skript gestoppt.")
    print("=" * 70)

    # --- SCHRITT 3: Fehlererkennung (Detect) ---
    print("\n--- Schritt 3: Fehlererkennung (Detect) ---")
    try:
        detectors = [NullDetector(), ViolationDetector()]
        print("Starte Fehlererkennung (detect_errors)...")
        t0 = time.time()
        hc.detect_errors(detectors)
        t1 = time.time()
        print(f"Fehlererkennung abgeschlossen. (Dauer: {t1 - t0:.2f} Sekunden)")
    except Exception as e:
        print(f"FEHLER bei der Fehlererkennung (detect_errors): {e}")
        sys.exit("Skript gestoppt.")

    # --- SCHRITT 4: Domänen-Setup & Reparatur ---
    print("\n--- Schritt 4: Domänen-Setup und Fehlerreparatur (Repair) ---")
    try:
        print("Starte Domänen-Setup (setup_domain)...")
        hc.setup_domain()
        print("Domänen-Setup abgeschlossen.")
    except Exception as e:
        print(f"FEHLER beim Domänen-Setup (setup_domain): {e}")
        sys.exit("Skript gestoppt.")

    try:
        featurizers = [InitAttrFeaturizer(), OccurAttrFeaturizer(), FreqFeaturizer(), ConstraintFeaturizer()]
        print("Starte Fehlerreparatur (repair_errors)... (kann einige Minuten dauern)")
        t0 = time.time()
        hc.repair_errors(featurizers)
        t1 = time.time()
        print(f"Fehlerreparatur abgeschlossen. (Dauer: {t1 - t0:.2f} Sekunden)")
    except Exception as e:
        print(f"FEHLER bei der Fehlerreparatur (repair_errors): {e}")
        sys.exit("Skript gestoppt.")
    print("=" * 70)

    # --- SCHRITT 5: Ergebnisse speichern ---
    print("\n--- Schritt 5: Ergebnisse (Reparierte Daten) speichern ---")
    reparierte_tabelle = f"{tabellename}_repaired"

    try:
        db_engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
        print(f"Datenbank-Engine für '{reparierte_tabelle}' erstellt.")
    except Exception as e:
        print(f"FEHLER beim Erstellen der SQLAlchemy-Engine: {e}")
        sys.exit("Skript gestoppt.")

    try:
        print(f"Lese reparierte Tabelle '{reparierte_tabelle}' aus der Datenbank...")
        df_sauber = pd.read_sql(f"SELECT * FROM {reparierte_tabelle}", db_engine)

        spalten = [c for c in df_sauber.columns if not c.startswith('_tid_')]
        print(f"Speichere reparierte Daten in '{ausgabedatei}'...")
        df_sauber.to_csv(ausgabedatei, index=False, columns=spalten)

        print("-" * 40)
        print(f"AD HOC: Zeilen im reparierten Datensatz: {len(df_sauber)}")
        print(f"ERGEBNIS: Reparierte Daten erfolgreich gespeichert: '{ausgabedatei}'")
        print("-" * 40)
    except Exception as e:
        print(f"FEHLER beim Lesen/Speichern der reparierten Tabelle: {e}")
        print(f"Bitte prüfen: Existiert Tabelle '{reparierte_tabelle}' in DB '{DB_NAME}'?")
        sys.exit("Skript gestoppt.")

    print("=" * 70)
    print("SKRIPT 07 (EXPERIMENT 2.1 – HoloClean) ERFOLGREICH ABGESCHLOSSEN.")
    print("=" * 70)


Lade HoloClean-Bibliotheken...
HoloClean-Module erfolgreich geladen.
Alle Bibliotheken sind bereit.
Starte HoloClean-Hauptprozess...
HoloClean-Session erfolgreich initialisiert (DB: 'holo').

--- Schritt 2: Daten laden (KeyError-Korrektur) ---
1) Lade Rohdaten aus 'rfd_main.csv' und entferne Spalten: ['url'] ...
2) Schreibe bereinigten Datensatz nach 'rfd_main_for_hc.csv'.
3) Lade Daten in Tabelle 'rfd_main' über HoloClean (load_data) ...


22:57:45 - [ INFO] - Loaded 1326 rows with 19890 cells
22:57:46 - [ INFO] - DONE Loading rfd_main_for_hc.csv


4) Lade Constraints (Regeln) aus 'rfd_constraints.txt' ...


22:57:46 - [ INFO] - DONE Loading DCs from rfd_constraints.txt


Daten (ohne 'url') und Constraints wurden erfolgreich geladen und verknüpft.

--- Schritt 3: Fehlererkennung (Detect) ---
Starte Fehlererkennung (detect_errors)...


...

Fehlererkennung abgeschlossen. (Dauer: 2.30 Sekunden)

--- Schritt 4: Domänen-Setup und Fehlerreparatur (Repair) ---
Starte Domänen-Setup (setup_domain)...


100%|██████████| 1326/1326 [00:00<00:00, 3390.12it/s]
0it [00:00, ?it/s]
574it [00:00, 5710.75it/s]0:00<?, ?it/s]
1141it [00:00, 5696.24it/s]00:00<00:01, 5959.17it/s]
1758it [00:00, 5815.75it/s][00:00<00:01, 5939.96it/s]
2294it [00:00, 5646.27it/s][00:00<00:00, 5997.36it/s]
2872it [00:00, 5676.30it/s][00:00<00:00, 5675.49it/s]
3478it [00:00, 5780.92it/s][00:00<00:00, 5783.05it/s]
4158it [00:00, 6004.03it/s][00:00<00:00, 5897.86it/s]
4831it [00:00, 6135.57it/s][00:00<00:00, 6067.90it/s]
5415it [00:00, 5595.00it/s][00:00<00:00, 6153.65it/s]
6022it [00:01, 5707.05it/s][00:00<00:00, 5593.40it/s]
6585it [00:01, 5601.10it/s][00:01<00:00, 5700.23it/s]
 91%|█████████▏| 6643/7262 [00:01<00:00, 5624.36it/s]
7262it [00:01, 5810.01it/s][00:01<00:00, 5908.90it/s]
22:57:54 - [ INFO] - number of (additional) weak labels assigned from posterior model: 2679
22:57:59 - [ INFO] - DONE with domain preparation.


Domänen-Setup abgeschlossen.
Starte Fehlerreparatur (repair_errors)... (kann einige Minuten dauern)


100%|██████████| 7262/7262 [00:00<00:00, 100368.19it/s]
22:58:05 - [ INFO] - DONE setting up featurized dataset.
22:58:05 - [ INFO] - DONE setting up repair model.
22:58:05 - [ INFO] - training with 6030 training examples (cells)
100%|██████████| 20/20 [01:10<00:00,  3.62s/it]
22:59:16 - [ INFO] - DONE training repair model.
22:59:16 - [ INFO] - inferring on 2519 examples (cells)
22:59:19 - [ INFO] - DONE inferring repairs.
22:59:19 - [ INFO] - DONE collecting the inferred values.
22:59:20 - [ INFO] - DONE generating repaired dataset


Fehlerreparatur abgeschlossen. (Dauer: 81.21 Sekunden)

--- Schritt 5: Ergebnisse (Reparierte Daten) speichern ---
Datenbank-Engine für 'rfd_main_repaired' erstellt.
Lese reparierte Tabelle 'rfd_main_repaired' aus der Datenbank...
Speichere reparierte Daten in 'ergebnisse/2.1_rfd_repaired_holoclean.csv'...
----------------------------------------
AD HOC: Zeilen im reparierten Datensatz: 1326
ERGEBNIS: Reparierte Daten erfolgreich gespeichert: 'ergebnisse/2.1_rfd_repaired_holoclean.csv'
----------------------------------------
SKRIPT 07 (EXPERIMENT 2.1 – HoloClean) ERFOLGREICH ABGESCHLOSSEN.
